# 🔬 CORTEX CONTROL — Mode Développement

**Frégate:** 00_CORTEX_HQ  
**Phase:** DELTA (Debug & Validation)  
**Version:** 2.0  
**Date:** 2026-02-02  

---

### 🎯 Objectifs
- Analyser et debugger une vidéo unique
- Tester différents prompts et modèles
- Inspecter les résultats en détail
- Valider avant scellage

### ⚠️ Usage
Ce notebook est destiné à la **phase de développement**. Pour la production de masse, utiliser `EXO_00_CORTEX_PRODUCTION.ipynb`.

In [ ]:
#@title CELLULE 2 — Installation & Imports
#@markdown Installe les dépendances et importe les librairies

!pip install google-generativeai opencv-python-headless -q

import os
import sys
import json
import cv2
from datetime import datetime
from IPython.display import display, Image, JSON
import google.generativeai as genai

# Vérification des versions
print("✅ Dépendances installées")
print(f"   - OpenCV: {cv2.__version__}")
print(f"   - google-generativeai: {genai.__version__}")

In [ ]:
#@title CELLULE 3 — Configuration
#@markdown Monte Google Drive et configure les chemins

from google.colab import drive
drive.mount('/content/drive')

# Configuration des chemins
DRIVE_ROOT = "/content/drive/MyDrive/DRIVE_EXODUS_V2"  #@param {type:"string"}
INPUT_DIR = f"{DRIVE_ROOT}/00_CORTEX_HQ/IN_VIDEO_SOURCE"
OUTPUT_DIR = f"{DRIVE_ROOT}/00_CORTEX_HQ/OUT_MANIFEST"
CODEBASE_DIR = f"{DRIVE_ROOT}/00_CORTEX_HQ/CODEBASE"

# Vérification des dossiers
for d in [INPUT_DIR, OUTPUT_DIR, CODEBASE_DIR]:
    if os.path.exists(d):
        print(f"✅ {d}")
    else:
        print(f"❌ Non trouvé: {d}")
        print(f"   → Création du dossier...")
        os.makedirs(d, exist_ok=True)

# Clé API (Colab Secrets ou input)
from google.colab import userdata
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    print("✅ Clé API chargée depuis Colab Secrets")
except:
    GEMINI_API_KEY = input("🔑 Entrez votre clé API Gemini: ")
    print("✅ Clé API saisie manuellement")

In [ ]:
#@title CELLULE 4 — Vérification API
#@markdown Teste la connexion à l'API Gemini

genai.configure(api_key=GEMINI_API_KEY)

try:
    models = list(genai.list_models())
    gemini_models = [m.name for m in models if 'gemini' in m.name.lower()]
    print("✅ API connectée avec succès!")
    print(f"\n📋 Modèles Gemini disponibles ({len(gemini_models)}):")
    for m in gemini_models[:10]:
        print(f"   - {m}")
    if len(gemini_models) > 10:
        print(f"   ... et {len(gemini_models) - 10} autres")
except Exception as e:
    print(f"❌ Erreur de connexion API: {e}")

In [ ]:
#@title CELLULE 5 — Sélection Vidéo
#@markdown Liste les vidéos disponibles et sélectionne celle à analyser

# Lister les vidéos disponibles
VIDEO_EXTENSIONS = ('.mp4', '.mov', '.avi', '.mkv', '.webm')
videos = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith(VIDEO_EXTENSIONS)] if os.path.exists(INPUT_DIR) else []

print(f"📁 Dossier: {INPUT_DIR}")
print(f"🎬 Vidéos trouvées ({len(videos)}):")
for i, v in enumerate(videos, 1):
    size_mb = os.path.getsize(f"{INPUT_DIR}/{v}") / (1024 * 1024)
    print(f"   {i}. {v} ({size_mb:.1f} MB)")

if not videos:
    print("⚠️ Aucune vidéo trouvée. Déposez vos vidéos dans IN_VIDEO_SOURCE/")

# Sélection manuelle
VIDEO_NAME = "source.mp4"  #@param {type:"string"}
video_path = f"{INPUT_DIR}/{VIDEO_NAME}"

if os.path.exists(video_path):
    print(f"\n✅ Vidéo sélectionnée: {VIDEO_NAME}")
else:
    print(f"\n❌ Vidéo non trouvée: {VIDEO_NAME}")

In [ ]:
#@title CELLULE 6 — Inspection Vidéo
#@markdown Affiche les métadonnées et un frame de preview

import tempfile
import base64
from IPython.display import HTML

if os.path.exists(video_path):
    cap = cv2.VideoCapture(video_path)

    # Métadonnées
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps if fps > 0 else 0
    file_size = os.path.getsize(video_path) / (1024 * 1024)

    print("📊 MÉTADONNÉES VIDÉO")
    print("=" * 40)
    print(f"Fichier:     {VIDEO_NAME}")
    print(f"Résolution:  {width}x{height}")
    print(f"FPS:         {fps:.2f}")
    print(f"Frames:      {frame_count}")
    print(f"Durée:       {duration:.2f}s ({int(duration//60)}m {int(duration%60)}s)")
    print(f"Taille:      {file_size:.2f} MB")
    print("=" * 40)

    # Extraire frame de preview (25%)
    preview_frame = int(frame_count * 0.25)
    cap.set(cv2.CAP_PROP_POS_FRAMES, preview_frame)
    ret, frame = cap.read()

    if ret:
        # Convertir BGR → RGB et encoder en JPEG
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        _, buffer = cv2.imencode('.jpg', frame_rgb)
        img_base64 = base64.b64encode(buffer).decode('utf-8')

        print(f"\n🖼️ Preview (frame {preview_frame}):")
        display(HTML(f'<img src="data:image/jpeg;base64,{img_base64}" width="640"/>'))

    cap.release()
else:
    print(f"❌ Vidéo non trouvée: {video_path}")

In [ ]:
#@title CELLULE 7 — Configuration Prompt (Éditable)
#@markdown Permet de modifier le prompt pour tests

USE_CUSTOM_PROMPT = False  #@param {type:"boolean"}

CUSTOM_PROMPT = """
Tu es un directeur de production expert en animation Roblox.
Analyse cette vidéo et génère un plan de production structuré.

IMPORTANT: Utilise UNIQUEMENT les IDs de l'Arsenal Impérial fournis.

Format de sortie JSON attendu:
{
  "metadata": { ... },
  "scenes": [ ... ],
  "production_notes": { ... }
}

Personnages valides: bacon_hair, noob, guest, builderman, robloxian_2_0
Props valides: linked_sword, firebrand, classic_jeep, wooden_chair, sparkles
Environnements valides: classic_baseplate, grass_terrain, city_street, medieval_castle
"""

if USE_CUSTOM_PROMPT:
    print("⚠️ Mode CUSTOM PROMPT activé")
    print("─" * 40)
    print(CUSTOM_PROMPT[:500] + "..." if len(CUSTOM_PROMPT) > 500 else CUSTOM_PROMPT)
else:
    print("✅ Utilisation du prompt standard (depuis EXO_00_CORTEX.py)")

In [ ]:
#@title CELLULE 8 — Sélection Modèle
#@markdown Choix du modèle Gemini à utiliser

MODEL_OPTIONS = [
    "gemini-2.5-flash-preview-05-20",
    "gemini-2.0-flash",
    "gemini-1.5-pro",
    "gemini-1.5-flash"
]

SELECTED_MODEL = "gemini-2.5-flash-preview-05-20"  #@param ["gemini-2.5-flash-preview-05-20", "gemini-2.0-flash", "gemini-1.5-pro", "gemini-1.5-flash"]

print(f"🤖 Modèle sélectionné: {SELECTED_MODEL}")
print("\n📋 Options disponibles:")
for m in MODEL_OPTIONS:
    marker = "→" if m == SELECTED_MODEL else " "
    print(f"   {marker} {m}")

In [ ]:
#@title CELLULE 9 — Exécution Analyse
#@markdown Lance l'analyse via le script principal

# Import du script principal
sys.path.insert(0, CODEBASE_DIR)

try:
    from EXO_00_CORTEX import (
        call_gemini,
        get_video_metadata,
        CortexLogger,
        SCENE_ANALYSIS_PROMPT
    )
    print("✅ Module EXO_00_CORTEX importé")
except ImportError as e:
    print(f"❌ Erreur d'import: {e}")
    print(f"   Vérifiez que EXO_00_CORTEX.py est dans: {CODEBASE_DIR}")
    raise

# Configuration environnement pour API
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

# Logger en mode verbose
logger = CortexLogger(verbose=True)

print("\n" + "=" * 60)
print("🚀 DÉMARRAGE ANALYSE CORTEX")
print("=" * 60)

# Obtenir métadonnées
metadata = get_video_metadata(video_path, logger)
print(f"\n📊 Métadonnées extraites: {json.dumps(metadata, indent=2)}")

# Analyse Gemini
print(f"\n🤖 Appel Gemini ({SELECTED_MODEL})...")
start_time = datetime.now()

result = call_gemini(
    video_path=video_path,
    metadata=metadata,
    logger=logger,
    model_name=SELECTED_MODEL
)

elapsed = (datetime.now() - start_time).total_seconds()
print(f"\n⏱️ Temps d'exécution: {elapsed:.2f}s")

if result:
    print("\n✅ Analyse terminée avec succès!")
else:
    print("\n❌ Échec de l'analyse")

In [ ]:
#@title CELLULE 10 — Inspection Résultat
#@markdown Affichage JSON formaté avec statistiques

if result:
    # Affichage JSON interactif
    print("📋 RÉSULTAT JSON (interactif):")
    display(JSON(result))

    # Statistiques rapides
    print("\n" + "=" * 60)
    print("📊 STATISTIQUES")
    print("=" * 60)

    scenes = result.get('scenes', result.get('scene_breakdown', []))
    print(f"Scènes:      {len(scenes)}")

    total_characters = 0
    total_props = 0
    all_characters = set()
    all_props = set()
    all_environments = set()

    for scene in scenes:
        chars = scene.get('characters', [])
        props = scene.get('props', [])
        env = scene.get('environment', {})

        total_characters += len(chars)
        total_props += len(props)

        for c in chars:
            if isinstance(c, dict):
                all_characters.add(c.get('character_id', 'unknown'))
            else:
                all_characters.add(str(c))

        for p in props:
            if isinstance(p, dict):
                all_props.add(p.get('prop_id', 'unknown'))
            else:
                all_props.add(str(p))

        if isinstance(env, dict):
            all_environments.add(env.get('environment_id', 'unknown'))

    print(f"Personnages: {total_characters} (uniques: {len(all_characters)})")
    print(f"Props:       {total_props} (uniques: {len(all_props)})")
    print(f"Environnements: {len(all_environments)}")

    print(f"\n🎭 Personnages: {', '.join(all_characters)}")
    print(f"🎪 Props: {', '.join(all_props)}")
    print(f"🏞️ Environnements: {', '.join(all_environments)}")

    # Notes de production
    notes = result.get('production_notes', {})
    if notes:
        print(f"\n📝 Notes de production:")
        print(f"   Complexité: {notes.get('complexity_score', 'N/A')}/10")
        print(f"   Temps estimé: {notes.get('estimated_render_hours', 'N/A')}h")
else:
    print("❌ Aucun résultat à afficher")

In [ ]:
#@title CELLULE 11 — Validation Arsenal
#@markdown Vérifie que tous les IDs sont dans l'Arsenal Impérial

try:
    from EXO_00_CORTEX import (
        VALID_CHARACTERS,
        VALID_PROPS,
        VALID_ENVIRONMENTS,
        VALID_ANIMATIONS,
        VALID_CAMERA_STYLES,
        VALID_LIGHTING_PRESETS,
        VALID_AUDIO
    )
except ImportError:
    print("⚠️ Impossible d'importer les constantes Arsenal")
    VALID_CHARACTERS = set()
    VALID_PROPS = set()
    VALID_ENVIRONMENTS = set()

if result:
    anomalies = []
    scenes = result.get('scenes', result.get('scene_breakdown', []))

    for i, scene in enumerate(scenes, 1):
        # Vérifier personnages
        for c in scene.get('characters', []):
            char_id = c.get('character_id') if isinstance(c, dict) else c
            if char_id and char_id not in VALID_CHARACTERS:
                anomalies.append(f"Scène {i}: Personnage invalide '{char_id}'")

        # Vérifier props
        for p in scene.get('props', []):
            prop_id = p.get('prop_id') if isinstance(p, dict) else p
            if prop_id and prop_id not in VALID_PROPS:
                anomalies.append(f"Scène {i}: Prop invalide '{prop_id}'")

        # Vérifier environnement
        env = scene.get('environment', {})
        env_id = env.get('environment_id') if isinstance(env, dict) else None
        if env_id and env_id not in VALID_ENVIRONMENTS:
            anomalies.append(f"Scène {i}: Environnement invalide '{env_id}'")

    print("🛡️ VALIDATION ARSENAL IMPÉRIAL")
    print("=" * 60)

    if anomalies:
        print(f"⚠️ {len(anomalies)} anomalie(s) détectée(s):")
        for a in anomalies:
            print(f"   - {a}")
        print("\n💡 Ces IDs seront auto-corrigés en 'generic_prop' par finalize_output()")
    else:
        print("✅ Tous les IDs sont conformes à l'Arsenal Impérial!")

    # Afficher l'Arsenal disponible
    print(f"\n📦 Arsenal disponible:")
    print(f"   Personnages: {len(VALID_CHARACTERS)}")
    print(f"   Props: {len(VALID_PROPS)}")
    print(f"   Environnements: {len(VALID_ENVIRONMENTS)}")
else:
    print("❌ Aucun résultat à valider")

In [ ]:
#@title CELLULE 12 — Sauvegarde Manuelle
#@markdown Sauvegarde le JSON avec timestamp

if result:
    # Nom du fichier
    video_base = VIDEO_NAME.rsplit('.', 1)[0]
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_name = f"PRODUCTION_PLAN_{video_base}_{timestamp}.json"
    output_path = f"{OUTPUT_DIR}/{output_name}"

    # Créer dossier si nécessaire
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Sauvegarder
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=2, ensure_ascii=False)

    print("💾 SAUVEGARDE")
    print("=" * 60)
    print(f"✅ Fichier sauvegardé:")
    print(f"   {output_path}")
    print(f"   Taille: {os.path.getsize(output_path) / 1024:.2f} KB")
else:
    print("❌ Aucun résultat à sauvegarder")

In [ ]:
#@title CELLULE 13 — Tests Unitaires
#@markdown Tests de validation de la structure JSON

def test_json_structure(data):
    """Valide la structure du PRODUCTION_PLAN.JSON"""
    errors = []

    # Vérifier metadata
    if "metadata" not in data:
        errors.append("Champ 'metadata' manquant")
    else:
        meta = data["metadata"]
        required_meta = ["source_video", "duration_seconds", "fps", "resolution"]
        for field in required_meta:
            if field not in meta:
                errors.append(f"metadata.{field} manquant")

    # Vérifier scenes
    scenes_key = "scenes" if "scenes" in data else "scene_breakdown"
    if scenes_key not in data:
        errors.append("Champ 'scenes' ou 'scene_breakdown' manquant")
    else:
        scenes = data[scenes_key]
        if not isinstance(scenes, list):
            errors.append("'scenes' doit être une liste")
        elif len(scenes) == 0:
            errors.append("'scenes' est vide")
        else:
            for i, scene in enumerate(scenes):
                if "scene_id" not in scene and "id" not in scene:
                    errors.append(f"Scene {i+1}: 'scene_id' manquant")

    # Vérifier production_notes (optionnel mais recommandé)
    if "production_notes" not in data:
        errors.append("[WARNING] Champ 'production_notes' manquant (optionnel)")

    return errors

def test_scene_completeness(data):
    """Vérifie que chaque scène a les champs essentiels"""
    errors = []
    scenes_key = "scenes" if "scenes" in data else "scene_breakdown"
    scenes = data.get(scenes_key, [])

    for i, scene in enumerate(scenes):
        if "timecode_start" not in scene:
            errors.append(f"Scene {i+1}: 'timecode_start' manquant")
        if "timecode_end" not in scene:
            errors.append(f"Scene {i+1}: 'timecode_end' manquant")
        if "characters" not in scene:
            errors.append(f"Scene {i+1}: 'characters' manquant")

    return errors

# Exécution des tests
if result:
    print("🧪 TESTS UNITAIRES")
    print("=" * 60)

    # Test 1: Structure JSON
    errors1 = test_json_structure(result)
    if errors1:
        print(f"\n❌ Test Structure: {len(errors1)} erreur(s)")
        for e in errors1:
            print(f"   - {e}")
    else:
        print("✅ Test Structure: PASSÉ")

    # Test 2: Complétude des scènes
    errors2 = test_scene_completeness(result)
    if errors2:
        print(f"\n❌ Test Complétude: {len(errors2)} erreur(s)")
        for e in errors2:
            print(f"   - {e}")
    else:
        print("✅ Test Complétude: PASSÉ")

    # Résumé
    total_errors = len([e for e in errors1 if not e.startswith("[WARNING]")]) + len(errors2)
    print("\n" + "=" * 60)
    if total_errors == 0:
        print("✅ TOUS LES TESTS PASSÉS")
    else:
        print(f"❌ {total_errors} ERREUR(S) DÉTECTÉE(S)")
else:
    print("❌ Aucun résultat à tester")